# 介入実験ノートブック

In [1]:
%load_ext autoreload
%autoreload 2
from mingpt.utils import set_seed
set_seed(44)

In [3]:
import os
import math
import time
import numpy as np
from copy import deepcopy
import pickle
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
from torch.nn import functional as F
from torch.utils.data import Subset
from tqdm import tqdm
from matplotlib import pyplot as plt

# Othello / Intervention 関連の関数群",
from data import get_othello, plot_probs, plot_mentals
from data.othello import permit, start_hands, OthelloBoardState, permit_reverse

# mingpt のモジュール",
from mingpt.dataset import CharDataset
from mingpt.model import GPT, GPTConfig, GPTforProbeIA
from mingpt.utils import sample, intervene, print_board
from mingpt.probe_model import BatteryProbeClassification, BatteryProbeClassificationTwoLayer

plt.rc('text', usetex=True)
plt.rc('font', **{'size': 14.0})
plt.rc('text.latex', preamble=r'\usepackage{lmodern}')

In [5]:
championship = False
id_dim = 128
how_many_history_step_to_use = 99
exp = f"state_tl{mid_dim}"
if championship:
    exp += "_championship"

# 介入設定
Load a game from intervention benchmark and select an intervention configuration

In [6]:
# 介入設定データのロード  
with open("intervention_benchmark.pkl", "rb") as input_file:
    dataset = pickle.load(input_file)

In [ ]:
case_id = 777
wtd = {
    "intervention_position": permit_reverse(dataset[case_id]["pos_int"]), 
    "intervention_from": dataset[case_id]["ori_color"], 
    "intervention_to": 2 - dataset[case_id]["ori_color"], 
}

print(wtd)

# ケースのゲーム履歴（手順）を抽出",
completion = dataset[case_id]["history"]

wtd_list = [wtd]

# プローブのロード

In [9]:
probes = {}
layer_s = 4
layer_e = 9
for layer in range(layer_s, layer_e):
    p = BatteryProbeClassificationTwoLayer(torch.cuda.current_device(), probe_class=3, num_task=64, mid_dim=mid_dim)
    load_res = p.load_state_dict(torch.load(f"./ckpts/battery_othello/{exp}/layer{layer}/checkpoint.ckpt"))
    p.eval()
    probes[layer] = p

# GPTモデルのロード

In [10]:

othello = get_othello(ood_perc=0., data_root=None, wthor=False, ood_num=1)
train_dataset = CharDataset(othello)

mconf = GPTConfig(61, 59, n_layer=8, n_head=8, n_embd=512)

models = {}
for layer in range(layer_s, layer_e):
    model = GPTforProbeIA(mconf, probe_layer=layer)
    # model = GPT(mconf)
    if championship:
        ckpt_path = f"./ckpts/gpt_championship.ckpt",
    else:
        ckpt_path = f"./ckpts/gpt_synthetic.ckpt",
    model.load_state_dict(torch.load(ckpt_path))
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        model = model.to(device)
        model.eval()
        models[layer] = model

100%|██████████| 1/1 [00:00<00:00, 36.48it/s]


Dataset created has 1 sequences, 61 unique words.


# ゲーム進行をチェック  
Check the partial game progression

In [ ]:
# 介入前の局面を生成
ab = OthelloBoardState()
ab.update(completion, prt=True)

In [12]:
# ゲーム履歴をトークン列に変換
partial_game = torch.tensor([train_dataset.stoi[s] for s in completion], dtype=torch.long).to(device)

# 介入前の合法手リストの取得  
Check pre-intervention ground-truth legal next-steps and the predicted ones

In [13]:
pre_intv_valids = [permit_reverse(_) for _ in ab.get_valid_moves()]
pre_intv_valids
print('Pre-intervention valid moves:', pre_intv_valids)

['b4', 'c6', 'd3', 'e7', 'f4', 'f6']

# 介入実験の実施

In [ ]:
# 1. 最初の興味のある層 (layer_s) における中間表現の取得",
p = probes[layer_s]
whole_mid_act = models[layer_s].forward_1st_stage(partial_game[None, :])  # [B, T, F=512]",
mid_act = whole_mid_act[0, -1]
pre_intv_logits = p(mid_act[None, :])[0].squeeze(0)  # [64, 3]",
plot_mentals(plt.figure(), pre_intv_logits)

In [ ]:
# # 介入前のラベルを取得して反復介入
labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)
new_mid_act = mid_act.clone()
for wtd in wtd_list:
    new_mid_act = intervene(p, new_mid_act, labels_pre_intv, wtd, htd={}, plot=True)
    pre_intv_logits = p(new_mid_act[None, :])[0].squeeze(0)  # [64, 3]
    labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)

In [ ]:
# 介入後の中間表現を元のテンソルに戻す
whole_mid_act[0, -1] = new_mid_act

In [ ]:
# 2. その後の各層（layer_s～layer_e-1）の伝播と介入の波及
for i, layer in enumerate(range(layer_s, layer_e - 1)):
    p = probes[layer+1]
    whole_mid_act = models[layer_s].forward_2nd_stage(whole_mid_act, layer, layer+1)[0]  # [1, T, F=512]",
    mid_act = whole_mid_act[0, -1]
    pre_intv_logits = p(mid_act[None, :])[0].squeeze(0)
    plot_mentals(plt.figure(), pre_intv_logits)
    labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)
    new_mid_act = mid_act.clone()
    for config in wtd_list:
        new_mid_act = intervene(p, new_mid_act, labels_pre_intv, config, htd={}, plot=True)
        pre_intv_logits = p(new_mid_act[None, :])[0].squeeze(0)
        labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)
        whole_mid_act[0, -1] = new_mid_act

# 介入後の予測とヒートマップの比較

In [14]:
# 介入前の予測 (heatmap 用に reshape してプロット)
pre_intv_pred, _ = model(partial_game[None, :])  # [B, T, F=512]
# no history activations used here, that's why the prediction map is different to across layers
pre_intv_pred = pre_intv_pred[0, -1, 1:]
padding = torch.zeros(2).cuda()
pre_intv_pred = torch.softmax(pre_intv_pred, dim=0)
pre_intv_pred = torch.cat([pre_intv_pred[:27], padding, pre_intv_pred[27:33], padding, pre_intv_pred[33:]], dim=0)

In [ ]:
fig=plt.figure(figsize=(10, 6), dpi= 80, facecolor='w', edgecolor='k')
vv = 0.2
sns.heatmap(pre_intv_pred.detach().cpu().numpy().reshape(8, 8), vmin=0., vmax=vv, 
            yticklabels=list("ABCDEFGH"), xticklabels=list(range(1,9)), square=True, 
            annot=True, fmt=".2f")

In [ ]:
# 介入後の予測   
tb_resumed = whole_mid_act
post_intv_pred, _ = models[layer_s].predict(tb_resumed)
post_intv_pred = post_intv_pred[0, -1, 1:]
post_intv_pred = torch.softmax(post_intv_pred, dim=0)
post_intv_pred = torch.cat([post_intv_pred[:27], padding, post_intv_pred[27:33], padding, post_intv_pred[33:]], dim=0)
fig2 = plt.figure(figsize=(10, 6))
sns.heatmap(post_intv_pred.detach().cpu().numpy().reshape(8, 8), annot=True, fmt='.2f')
plt.title('Post-intervention Prediction Heatmap')
plt.show()

# Attribution Plot via Intervention

In [ ]:
# 以下は、介入実験済みの局面に対して，全ての合法手に関する Attribution (寄与度) ヒートマップをプロットする例


In [ ]:
# 以下は、介入実験済みの局面に対して，全ての合法手に関する Attribution (寄与度) ヒートマップをプロットする例
tbu = pre_intv_valids
total = len(tbu)
rows = math.ceil(total / 4)

fig, axs = plt.subplots(rows, 4, figsize=(30, rows * 8), dpi= 80, facecolor='w', edgecolor='k')
axs = axs.flat

# gems には介入実験の結果（例: 介入前 'pre' の予測結果など）を格納していると仮定
gems = {}
gems["pre"] = pre_intv_pred.detach().cpu().numpy().reshape(8, 8)

for i, tobe_tcaved in enumerate(tbu):
    pred = permit(tobe_tcaved)  # 0-63
    r_pred, c_pred = pred // 8, pred % 8

    tbp = np.zeros((8, 8), )
    pre = gems["pre"]
    for k, w in gems.items():
        if k == "pre":
            continue
        move = permit(k)  # 0-63
        r, c = move // 8, move % 8
        tbp[r, c] = - w[r_pred, c_pred] + pre[r_pred, c_pred]
    ab.plot_hm(axs[i], tbp.flatten(), permit(tobe_tcaved), logit=True)

plt.suptitle('Attribution via Intervention Heatmaps')
plt.show()

## Check post-intervention ground-truth next steps
0 for white; 1 for blank; 2 for black

In [16]:
htd = {"lr": 1e-3, "steps": 1000, "reg_strg": 0.2}
for wtd in wtd_list:
    move = permit(wtd["intervention_position"])
    r, c = move // 8, move % 8
    ab.state[r, c] = wtd["intervention_to"] - 1
ab.__print__()
post_intv_valids = [permit_reverse(_) for _ in ab.get_valid_moves()]
post_intv_valids

--------------------
['e6', 'd6', 'c3', 'f3', 'c4']
a                
b                
c     X X        
d       X X O    
e       O X O    
f     O          
g                
h                
  1 2 3 4 5 6 7 8
--------------------


['b3', 'b4', 'c6', 'd3', 'f4']

## Intervene and observe how the world representation changes along the way

In [ ]:
fig, axs = plt.subplots(layer_e - layer_s, 2, figsize=(8 * (1), 8 * (layer_e - layer_s)), dpi= 80, facecolor='w', edgecolor='k')
# two rows for the intervened layer layer_s, one for the rest
if len(axs.shape) == 1:
    axs = axs[:, None]

p = probes[layer_s]
whole_mid_act = models[layer_s].forward_1st_stage(partial_game[None, :])  # [B, T, F=512]

# intervene at the earlest interested layer 
mid_act = whole_mid_act[0, -1]
pre_intv_logits = p(mid_act[None, :])[0].squeeze(0)  # [64, 3]
plot_mentals(axs[0, 0], pre_intv_logits)
axs[0, 0].set_title(f"Pre-intervention Probe Result \n at the {layer_s}-th Layer")
labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)
new_mid_act = mid_act.clone()
for wtd in wtd_list:
    new_mid_act = intervene(p, new_mid_act, labels_pre_intv, wtd, htd, plot=True)
    pre_intv_logits = p(new_mid_act[None, :])[0].squeeze(0)  # [64, 3]
    labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)
post_intv_logits = p(new_mid_act[None, :])[0].squeeze(0)  # [64, 3]
plot_mentals(axs[0, 1], post_intv_logits)
axs[0, 1].set_title(f"Post-intervention Probe Result \n at the {layer_s}-th Layer")
# swap in 
whole_mid_act[0, -1] = new_mid_act

for i, layer in enumerate(range(layer_s, layer_e - 1)):  # 4, 5, 6, 7, indices of the layers to be passed
    p = probes[layer+1]
    whole_mid_act = models[layer_s].forward_2nd_stage(whole_mid_act, layer, layer+1)[0]  # [1, T, F=512]
    
    # intervene the output of the features freshly out
    mid_act = whole_mid_act[0, -1]
    pre_intv_logits = p(mid_act[None, :])[0].squeeze(0)  # [64, 3]
    plot_mentals(axs[i+1, 0], pre_intv_logits)
    axs[i+1, 0].set_title(f"Post-intervention Probe Result \n at the {layer+1}-th Layer")
    labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)
    new_mid_act = mid_act.clone()
    for wtd in wtd_list:
        new_mid_act = intervene(p, new_mid_act, labels_pre_intv, wtd, htd, plot=True)
        pre_intv_logits = p(new_mid_act[None, :])[0].squeeze(0)  # [64, 3]
        labels_pre_intv = pre_intv_logits.detach().argmax(dim=-1)
    post_intv_logits = p(new_mid_act[None, :])[0].squeeze(0)  # [64, 3]
    plot_mentals(axs[i+1, 1], post_intv_logits)
    axs[i+1, 1].set_title(f"Post-intervention Probe Result \n at the {layer+1}-th Layer")
    # swap in 
    whole_mid_act[0, -1] = new_mid_act

## Compare post-intervention prediction heatmap with pre-intervention ones
Underscored tiles are the ground-truth